In [1]:
import sqlite3

In [2]:
con = sqlite3.connect(':memory:')
cur = con.cursor()

In [3]:
cur.execute('CREATE TABLE temp (name, age)')

In [4]:
cur.execute('INSERT INTO temp (name, age) VALUES (1,2)')

In [5]:
cur.execute('SELECT * FROM temp')

In [6]:
cur.fetchall()

[(1, 2)]

In [7]:
# qmark, named 스타일로 전달을 해보자
cur.execute('INSERT INTO temp VALUES(?,?)', ['hi', 'hello'])

In [8]:
cur.execute('SELECT * FROM temp')
cur.fetchall()

[(1, 2), ('hi', 'hello')]

In [9]:
cur.execute('INSERT INTO temp VALUES(:name, :age)',
           {'name':'이름', 'age':4})

In [10]:
cur.execute('SELECT * FROM temp')
cur.fetchall()

[(1, 2), ('hi', 'hello'), ('이름', 4)]

In [12]:
data = [(1, 'A'), (2, 'B'), (3, 'C')]
cur.executemany('INSERT INTO temp VALUES(?,?)', data)
cur.execute('SELECT * FROM temp')
cur.fetchall()

[(1, 2), ('hi', 'hello'), ('이름', 4), (1, 'A'), (2, 'B'), (3, 'C')]

In [14]:
data = [{'name':4, 'age':'A'}, {'name':5, 'age':'B'}, {'name':6, 'age':'C'}]
cur.executemany('INSERT INTO temp VALUES(:name, :age)', data)
cur.execute('SELECT * FROM temp')
cur.fetchall()

[(1, 2),
 ('hi', 'hello'),
 ('이름', 4),
 (1, 'A'),
 (2, 'B'),
 (3, 'C'),
 (4, 'A'),
 (5, 'B'),
 (6, 'C')]

In [15]:
cur.execute('SELECT * FROM temp')
cur.fetchmany(4)

[(1, 2), ('hi', 'hello'), ('이름', 4), (1, 'A')]

In [16]:
cur.execute('SELECT * FROM temp LIMIT 1,4')
cur.fetchall()

[('hi', 'hello'), ('이름', 4), (1, 'A'), (2, 'B')]

In [26]:
cur.execute('DROP TABLE temp2')

In [27]:
cur.executescript('''
    CREATE TABLE temp2 (
        pk INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT
    );
    
    INSERT INTO temp2 (name) VALUES('이름1');--   => pk X 자동 1
    INSERT INTO temp2 VALUES (NULL, '이름2');--   => null 자동 2
''')

In [28]:
cur.execute('SELECT * FROM temp2')
cur.fetchall()

[(1, '이름1'), (2, '이름2')]

In [29]:
cur.execute('INSERT INTO temp2 VALUES(1,1)')

IntegrityError: UNIQUE constraint failed: temp2.pk

In [44]:
# DB 닫기
con.close()

In [45]:
con = sqlite3.connect('test.db')
cur = con.cursor()

In [46]:
cur.executescript('''
    DROP TABLE IF EXISTS temp1;
    CREATE TABLE temp1 (
        pk INTEGER PRIMARY KEY,
        name TEXT
    );
    
    INSERT INTO temp1 VALUES(1, '이름1');
    INSERT INTO temp1 VALUES(2, '이름2');
    
''')

In [47]:
cur.execute('SELECT * FROM temp1')
cur.fetchall()

[(1, '이름1'), (2, '이름2')]

# DB Browser for SQLite3 현재 쥬피터 파일 있는 곳에 생선된 test DB를 저 앱으로 열서어 확인

In [49]:
cur.execute("INSERT INTO temp1 (name) VALUES('111')")

In [50]:
cur.execute("INSERT INTO temp1 VALUES(NULL, '보이나')")
# 실핼 후 브라우저로 확인 (값 들어있는지 확인)

In [51]:
# 이걸 돌려도 물리적인 db에 보이지 않는다.
cur.execute('SELECT * FROM temp1')
cur.fetchall()

[(1, '이름1'), (2, '이름2'), (3, '111'), (4, '보이나')]

In [52]:
# commit을 해야 db에 보인다
con.commit()

In [53]:
# insert 된 pk값
cur.lastrowid

4

# 아래 코드 놓침 하나 한개

In [54]:
cur.execute('SELECT * FROM temp1 WHERE pk=usernan')
cur.fetchall()

OperationalError: no such column: usernan

In [55]:
con.close()

In [56]:
### 예제 _ 프랜차이즈 전산실
# CITY, SUPPLIER, PART, SELLS
# CITY: PK, NAME
# SUPPLIER; PK, NAME, FK(CITY.PK)
# PARK: PK, NAME
# SELLES: FK(S.PK), FK(P.PK), PRICE

In [57]:
con = sqlite3.connect('test1.db')
cur = con.cursor()

In [58]:
cur.executescript('''
    create table city (
        pk integer primary key,
        name text
    );
    create table supplier (
        pk integer primary key,
        name text,
        fk integer not null
    );
    create table part (
        pk integer primary key,
        name text
    );
    create table sells (
        fk1 integer not null,
        fk2 integer not null,
        price integer not null
    );
''')

In [59]:
data = [('성북구',), ('중구',), ('강북구',), ('어쩌구',), ('저쩌구',)]
cur.executemany('INSERT INTO city(name) values(?)', data)

In [60]:
cur.execute('SELECT * FROM city')
cur.fetchall()

[(1, '성북구'), (2, '중구'), (3, '강북구'), (4, '어쩌구'), (5, '저쩌구')]

In [61]:
data = [('메뉴1',), ('메뉴2',), ('메뉴3',), ('메뉴4',), ('메뉴5',)]
cur.executemany('INSERT INTO part(name) values(?)', data)

In [62]:
cur.execute('SELECT * FROM part')
cur.fetchall()

[(1, '메뉴1'), (2, '메뉴2'), (3, '메뉴3'), (4, '메뉴4'), (5, '메뉴5')]

In [65]:
cur.execute('SELECT * FROM city')
for row in cur.fetchall():
    if row[1] == '성북구':
        print(row[0])

1


In [70]:
cur.execute('SELECT pk FROM city WHERE name=:name', {'name':'성북구'})
cur.fetchone()[0]

1

In [72]:
cur.execute('SELECT pk FROM city WHERE name LIKE :name', {'name':'%강북%'})
cur.fetchone()[0]

3

In [73]:
cur.execute('SELECT * FROM (SELECT pk FROM city WHERE name LIKE :name)', {'name':'%강북%'})
cur.fetchall()

[(3,)]

In [75]:
cur.execute('SELECT * FROM city')
for row in cur.fetchall():
    if row[1] == '성북구':
        pk = row[0]
        cur.execute('INSERT INTO supplier(name, fk) VALUES(?,?)',
                   ['안암1호점', pk])
        break

In [78]:
cur.execute('SELECT * FROM supplier')
cur.fetchall()

[(1, '안암1호점', 1)]

In [79]:
cur.execute('SELECT pk FROM city WHERE name LIKE :name', {'name':'%성북%'})
pk = cur.fetchone()[0]
cur.execute('INSERT INTO supplier(name, fk) VALUES(?,?)',
           ['안암2호점', pk])

In [80]:
cur.execute('SELECT * FROM supplier')
cur.fetchall()

[(1, '안암1호점', 1), (2, '안암2호점', 1)]

In [81]:
cur.execute('''
    insert into supplier(name, fk)
    values(?, (select pk from city where name like ? limit 0,1))
''', ['종암1호점', '%성북%'])

In [82]:
cur.execute('SELECT * FROM supplier')
cur.fetchall()

[(1, '안암1호점', 1), (2, '안암2호점', 1), (3, '종암1호점', 1)]

In [84]:
con.commit()

In [89]:
data = ['%안암1호%', '%메뉴1%', 4500]

cur.execute('''
    insert into sells(fk1, fk2, price)
    values((select pk from supplier where name like ? limit 0,1),
            (select pk from part where name like ? limit 0,1),
            ?)
''', data)

In [92]:
data = [['%안암1호%', '%메뉴2%', 4500],
       ['%안암1호%', '%메뉴2%', 5000],
       ['%안암2호%', '%메뉴1%', 4700],
       ['%안암2호%', '%메뉴2%', 4900],
       ['%안암1호%', '%메뉴4%', 5300],
       ['%안암1호%', '%메뉴5%', 2300],
       ['%안암1호%', '%메뉴1%', 900]]

cur.executemany('''
    insert into sells(fk1, fk2, price)
    values((select pk from supplier where name like ? limit 0,1),
            (select pk from part where name like ? limit 0,1),
            ?)
    ''', data)

In [93]:
cur.execute('SELECT * FROM sells')
cur.fetchall()

[(1, 1, 4500),
 (1, 2, 4500),
 (1, 2, 5000),
 (2, 1, 4700),
 (2, 2, 4900),
 (1, 4, 5300),
 (1, 5, 2300),
 (1, 1, 900)]

In [95]:
cur.execute('''
    select city.name, supplier.name from city
    inner join supplier
    on city.pk = supplier.fk
    ''')
cur.fetchall()

[('성북구', '안암1호점'), ('성북구', '안암2호점'), ('성북구', '종암1호점')]